In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
import re
from pathlib import Path

# Folder with the runs
runs_folder = 'runs/'

# Get folder for each lambda
runs_folder = Path(runs_folder)
subfolders = [item for item in runs_folder.iterdir() if item.is_dir()]
lambda_folders = sorted([item.name for item in runs_folder.iterdir() if item.is_dir()])
lambdas = [re.findall(r'\d*\.?\d+', lambda_folder)[-1] for lambda_folder in lambda_folders]

# Load the csv for each lambda and each seed as a Pandas array
return_per_timestep_for_each_lambda = pd.DataFrame(columns=lambdas)
for lambda_folder in lambda_folders:
    print(f'Reading folder: {runs_folder/lambda_folder}')
    
    # Get lambda value
    labas = re.findall(r'\d*\.?\d+', lambda_folder)[-1]

    # Getting all the csv files in folder
    same_lambda_different_seeds_csvs = [e for e in (runs_folder/lambda_folder).iterdir() if str(e).endswith('.csv')]
    number_of_seeds = len(same_lambda_different_seeds_csvs)

    # Interpolate and average different seeds
    averaged_interpolated_returns = np.zeros(500_000+1)
    for csv_path in same_lambda_different_seeds_csvs:
        # Reading CSV
        dataframe = pd.read_csv(csv_path)
        if 'charts/episodic_return'in list(dataframe.columns):
            dataframe.rename(columns={'charts/episodic_return':'episodic_return'}, inplace=True)

        dataframe = dataframe[['episodic_return','global_step']]
        dataframe = dataframe.dropna()
        # Interpolation
        x_col = 'global_step'    
        y_col = 'episodic_return'

        interpolated_values = np.interp(np.linspace(0, 500_000-1, 500_000+1), dataframe[x_col], dataframe[y_col])

        # Averaging
        averaged_interpolated_returns += interpolated_values/number_of_seeds
        if np.isnan(interpolated_values.mean()):
            print(csv_path)
    # Saving 
    return_per_timestep_for_each_lambda[labas] = averaged_interpolated_returns

Reading folder: runs/lambda_0
Reading folder: runs/lambda_0.1
Reading folder: runs/lambda_0.2
Reading folder: runs/lambda_0.3
Reading folder: runs/lambda_0.4
Reading folder: runs/lambda_0.5
Reading folder: runs/lambda_0.6
Reading folder: runs/lambda_0.7
Reading folder: runs/lambda_0.8
Reading folder: runs/lambda_0.9
Reading folder: runs/lambda_0.905
Reading folder: runs/lambda_0.91
Reading folder: runs/lambda_0.915
Reading folder: runs/lambda_0.92
Reading folder: runs/lambda_0.925
Reading folder: runs/lambda_0.93
Reading folder: runs/lambda_0.935
Reading folder: runs/lambda_0.94
Reading folder: runs/lambda_0.945
Reading folder: runs/lambda_0.95
Reading folder: runs/lambda_0.955
Reading folder: runs/lambda_0.96
Reading folder: runs/lambda_0.965
Reading folder: runs/lambda_0.97
Reading folder: runs/lambda_0.975
Reading folder: runs/lambda_0.98
Reading folder: runs/lambda_0.985
Reading folder: runs/lambda_0.99
Reading folder: runs/lambda_0.995
Reading folder: runs/lambda_1


In [54]:
import matplotlib.pyplot as plt

y = return_per_timestep_for_each_lambda.mean(axis=0)
x = np.array(return_per_timestep_for_each_lambda.columns, dtype=np.float32)
fig = px.line(x=x, y=y)
fig.update_layout(title='Area under curve (AUC)', xaxis_title='λ value', yaxis_title='Area Under Curve (AUC)')
fig.show()

In [45]:
return_per_timestep_for_each_lambda.columns = np.array(return_per_timestep_for_each_lambda.columns, dtype=np.float64)

In [51]:
px.line(return_per_timestep_for_each_lambda[[0,0.1,0.2,1]][::1000])